<a href="https://colab.research.google.com/github/dameem4/new-project/blob/master/Kasali_CSC_40098_Disseration_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Table of Contents

<details><summary>
<font color='Blue'> I. Installation of MMF & dependencies </font></summary>

- Install MMF from source
</details>

<details><summary>
<font color='Blue'> II. Download the datasets & convert them into MMF format </font></summary>


</details>


<details><summary>
<font color='Blue'> III. Feature Extraction </font></summary>

</details>

<details><summary>
<font color='Blue'> IV. Fine-tuning pre-trained VisualBERT models on Hateful Memes </font></summary>

</details>


<details><summary>
<font color='Blue'> V. Generate predictions for the Challenge (`test_unseen.jsonl`) </font></summary>

</details>


## <font color='green'> <b> I. Installation of MMF & dependencies </b> </font>

Please set your `$HOME` directory.\
**e.g.** For *Linux* users it can be: `"/home"`,\
For *Colab* it would be: `"/content"`

In [ ]:
import os
home = "/content"
os.chdir(home)
os.getcwd()

'/content'

In [ ]:
# Install specified versions of `torch` and `torchvision`, before installing mmf (causes an issue)
!pip install torch torchvision -f https://download.pytorch.org/whl/torch_stable.html

#### *Install MMF from source* 


In [ ]:
# Clone the following repo where mmf does not install default image features, 
# since we will use our own features
!git clone https://github.com/facebookresearch/mmf.git
%cd /content/mmf
!pip install --editable .

In [ ]:
!rm -rf mmf

In [ ]:
os.chdir(os.path.join(home, "mmf"))

---
## <font color='green'> <b> II. Download the datasets & convert them into *MMF* format </b> </font> <font color='red'><b> --Action required!-- </b></font>

### <font color='Orchid'> <b> Hateful Memes  dataset </b> </font>

Please download the `Hateful Memes Dataset` from the official challenge webpage: https://hatefulmemeschallenge.com/#download

After filling the form the `hateful_memes.zip` file will be downloaded, which includes all the required data including images. Please define the variable `PATH_TO_ZIP_FILE` in the following code cell which stores the full path of the downloaded `.zip` file:


In [ ]:
PATH_TO_ZIP_FILE = "/content/drive/Othercomputers/MyLaptop/memes/hateful_memes.zip"
!cp -r $PATH_TO_ZIP_FILE /content/mmf/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mmf

In [ ]:
# Add the mmf folder to Python Path
os.environ['PYTHONPATH'] += ":/content/mmf/"

In [ ]:
!mmf_convert_hm --zip_file="hateful_memes.zip" --password DontTellYou

In [ ]:
!pip install mmf

In [ ]:
# Check how many images we have in total
!ls /root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/img/ | wc -l

That means there are `12.140` **'uniquely named'** images in total and you might recall that the sizes of each set was the following:

- `|train.jsonl| = 8.500`
- `|dev_seen.jsonl| = 500`
- `|dev_unseen.jsonl| = 540`
- `|test_seen.jsonl| = 1.000`
- `|test_unseen.jsonl| = 2.000`

Well, this makes `8.500 + 500 + 540 + 1.000 + 2.000 = 12.540` in total. \
> *Is there something wrong?*\
> **TL;DR:** Nope. Some images in `dev_seen` are used in `dev_unseen`, too. To be specific, they have `400` common images. Hence, in total we have `12.540 - 400 = 12.140` *'unique'* images.\
See <font color='orange'> <b> Extras </b> </font> --> <font color='Gold'><b> Number of 'unique' (based on file names) images </b></font> at the end of this script to see the explanation in detail.

In [ ]:
# Free up the disk by removing .zip, .tar files
!rm -rf /root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/hateful_memes.zip
!rm -rf $home/mmf/hateful_memes.zip

### <font color='Orchid'> <b> Memotion dataset </b> </font>

There are 2 options for downloading the dataset: 
1. download the dataset (a `.zip` file) using `Kaggle API`\
OR
2. download the dataset (a `.zip` file) from [Kaggle](https://www.kaggle.com/williamscott701/memotion-dataset-7k) directly, **(<font color='red' >preferred </font> if you're not familiar with Kaggle API)**

#### <font color='Thistle'> <b> 1. Download Memotion dataset using `Kaggle API` </b> </font>

Check out the official documentation to get more information on Kaggle API and how to create a Kaggle API Key:
- [Link#1](https://github.com/Kaggle/kaggle-api#api-credentials)
- [Link#2](https://www.kaggle.com/docs/api)

The API Key is stored in a file named `kaggle.jsonl`, which has the folowing line inside: 
`{"username":"your_user_name","key":"some_values_here"}`

> Upload the `kaggle.json` file to your `$HOME` directory and run the following cell.

In [ ]:
# Install kaggle library
!pip install -q kaggle
# Create a directory where API key will be stored
!mkdir -p ~/.kaggle
# Move the API key to where Kaggle expects it to be
!mv $home/kaggle.json ~/.kaggle/
# Give according rights to the file
!chmod 600 /root/.kaggle/kaggle.json
# Finally, download the dataset (.zip file)
!kaggle datasets download -d williamscott701/memotion-dataset-7k
# Unzip the data 
!unzip -qq memotion-dataset-7k.zip -d $home/

#### <font color='Thistle'> <b> 2. Download Memotion dataset directly from [Kaggle](https://www.kaggle.com/williamscott701/memotion-dataset-7k) </b> </font>

Download the dataset and put the `.zip` file into your `$HOME` directory and then run the following cell:

In [ ]:
# Unzip the data 
# !unzip memotion-dataset-7k.zip -d $home/

#### <font color='Thistle'> <b> Labeling Memotion Dataset </b> </font>

We have added `Memotion Dataset` to `Hateful Memes Dataset` and fine-tuned some models on the *aggregated* data. But there was no significant improvement seen neither on the `ROC-AUC score`, nor on the `accuracy`. We then discovered that the dataset is *horribly* labeled. Therefore, one needs to label the dataset.

So we went through the dataset and cherry-picked the memes that would be suitable for the challenge, considering the idea of `Hateful Memes Challenge`.

The following cell can be run to clone a repository which includes helpful scripts for the project such as; a script for labeling the `Memotion Dataset` and saving the data in the same format as the `Hateful Memes Dataset`.

In [ ]:
os.chdir(home)
!git clone https://github.com/facebookresearch/detectron2.git

Cloning into 'hateful_memes-hate_detectron'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 47 (delta 19), reused 43 (delta 15), pack-reused 0
Unpacking objects: 100% (47/47), done.


Labeling the dataset is not necessary for reproducing our results but one can check out the [labeling script](https://github.com/rizavelioglu/hateful_memes-hate_detectron/tree/main/utils/label_memotion.py) and execute the following line of code to run the script and see how the labeling is done.

```
# Start labeling Memotion dataset and save it at the end
%run $home/hateful_memes-hate_detectron/utils/label_memotion.py --home $home
```

---
> In total, we have labeled $328$ memes.\
Check out the following file to find the ones we labeled: [/hateful_memes-hate_detectron/utils/label_memotion.jsonl](https://github.com/rizavelioglu/hateful_memes-hate_detectron/tree/main/utils/label_memotion.jsonl)

Next, we move those labeled images from `Memotion Dataset` into the same folder where the images from `Hateful Memes Dataset` are, so that when the image features are being extracted all the images are inside the same folder.


In [ ]:
import pandas as pd
# read the .jsonl file and get the img column
labeled_memo_samples = pd.read_json(os.path.join(home, "hateful_memes-hate_detectron/utils/label_memotion.jsonl"), lines=True)['img']
# parse the img entries and get the image names
labeled_memo_samples = [i.split('/')[1] for i in list(labeled_memo_samples)]

img_dir = os.path.join(home, f"memotion_dataset_7k/images/")
for img in labeled_memo_samples:
    os.rename(f"{img_dir+img}", f"/root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/img/{img}")

In [ ]:
# Check how many images we have in total
!ls /root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/img/ | wc -l

12468


### <font color='Orchid'> <b> Merging the two datasets to get a larger training data</b> </font>


Simply execute the following cell which concatanates;
- Labeled Memotion dataset,
- Hateful Memes' training data, and
- 100 images from `dev_seen.jsonl` that are not in `dev_unseen.jsonl`

and generates `train_v10.jsonl`, which will be used for fine-tuning.

In [ ]:
!python $home/hateful_memes-hate_detectron/utils/concat_memotion-hm.py --home $home

---
## <font color='green'> <b> III. Feature Extraction </b> </font>

### <font color='lightgreen'> <b> Extract image features using [`mmf/tools/scripts/features/extract_features_vmb.py`](https://github.com/facebookresearch/mmf/blob/master/tools/scripts/features/extract_features_vmb.py) </b> </font>

#### Install packages & repos

In [ ]:
import os
os.chdir(home)
!git clone https://gitlab.com/vedanuj/vqa-maskrcnn-benchmark

In [ ]:
!pip install ninja yacs cython matplotlib

In [ ]:
os.chdir(os.path.join(home, "vqa-maskrcnn-benchmark"))
!rm -rf build
!python setup.py build develop

#### Extract!

In [ ]:
# !wget https://dl.fbaipublicfiles.com/pythia/detectron_model/FAST_RCNN_MLP_DIM2048_FPN_DIM512.pkl
# !wget https://dl.fbaipublicfiles.com/pythia/detectron_model/e2e_faster_rcnn_X-101-64x4d-FPN_1x_MLP_2048_FPN_512.yaml
os.chdir(os.path.join(home, "mmf/tools/scripts/features/"))
out_folder = os.path.join(home, "features/")

!python extract_features_vmb.py --config_file "https://dl.fbaipublicfiles.com/pythia/detectron_model/detectron_model_x152.yaml" \
                                --model_name "X-152" \
                                --output_folder $out_folder \
                                --image_dir "/root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/img/" \
                                --num_features 100 \
                                # --exclude_list "/content/exclude.txt"
                                # --feature_name "fc6" \
                                # --confidence_threshold 0. \

---
## <font color='green'> <b> IV. Fine-tuning pre-trained VisualBERT models on Hateful Memes </b> </font>

*italicised text*### <font color='Violet'> <b> Fine tuning  </b> </font>

In [ ]:
"""
Uncomment it if needed
"""

# os.chdir(home)
# # Define where image features are
# feats_dir = os.path.join(home, "features")
# # Define where train.jsonl is
# train_dir = os.path.join(home, "train_v9.jsonl")

# !mmf_run config="projects/visual_bert/configs/hateful_memes/from_coco.yaml" \
#         model="visual_bert" \
#         dataset=hateful_memes \
#         run_type=train_val \
#         checkpoint.max_to_keep=1 \
#         checkpoint.resume_zoo=visual_bert.pretrained.cc.full \
#         training.tensorboard=True \
#         training.checkpoint_interval=50 \
#         training.evaluation_interval=50 \
#         training.max_updates=3000 \
#         training.log_interval=100 \
#         dataset_config.hateful_memes.max_features=100 \
#         dataset_config.hateful_memes.annotations.train[0]=$train_dir \
#         dataset_config.hateful_memes.annotations.val[0]=hateful_memes/defaults/annotations/dev_unseen.jsonl \
#         dataset_config.hateful_memes.annotations.test[0]=hateful_memes/defaults/annotations/test_unseen.jsonl \
#         dataset_config.hateful_memes.features.train[0]=$feats_dir \
#         dataset_config.hateful_memes.features.val[0]=$feats_dir \
#         dataset_config.hateful_memes.features.test[0]=$feats_dir \
#         training.lr_ratio=0.3 \
#         training.use_warmup=True \
#         training.batch_size=32 \
#         optimizer.params.lr=5.0e-05 \
#         env.save_dir=./sub1 \
#         env.tensorboard_logdir=logs/fit/sub1 \

##### **Visualize losses/accuracy via Tensorboard**

In [ ]:
# Load the TensorBoard notebook extension
# %load_ext tensorboard

In [ ]:
# %tensorboard --logdir logs/fit

---
## <font color='green'> <b> V. Generate predictions for the Challenge (`test_unseen.jsonl`) </b> </font>

*italicised text*### <font color='Thistle'> <b> Testing Phase 1 </b> </font>

In [ ]:
"""
Uncomment it if needed
"""

# os.chdir(home)
# # where checkpoint is
# ckpt_dir = os.path.join(home, "sub1/best.ckpt")
# feats_dir = os.path.join(home, "features/feats_hm")

# !mmf_predict config="projects/visual_bert/configs/hateful_memes/defaults.yaml" \
#     model="visual_bert" \
#     dataset=hateful_memes \
#     run_type=test \
#     checkpoint.resume_file=$ckpt_dir \
#     checkpoint.reset.optimizer=True \
#     dataset_config.hateful_memes.annotations.val[0]=hateful_memes/defaults/annotations/dev_unseen.jsonl \
#     dataset_config.hateful_memes.annotations.test[0]=hateful_memes/defaults/annotations/test_unseen.jsonl \
#     dataset_config.hateful_memes.features.train[0]=$feats_dir \
#     dataset_config.hateful_memes.features.val[0]=$feats_dir \
#     dataset_config.hateful_memes.features.test[0]=$feats_dir \


### <font color='Gold'> <b> Image feature type conversion </b> </font>
Convert image features from `.npy` --> `.lmdb` and vice versa

You can also try to use the .npy files directly. Just point to the folder which contains those files in your config. lmdb is not a necessary requirement.

#### <font color='PaleGoldenrod'> <b> Convert .npy files to .lmdb </b> </font>

In [ ]:
import argparse
import glob
import os
import pickle

import lmdb
import numpy as np
import tqdm


class LMDBConversion():
    def __init__(self, features_folder, lmdb_path):
        self.features_folder = features_folder
        self.lmdb_path = lmdb_path

    def convert(self):
        env = lmdb.open(self.lmdb_path, map_size=1099511627776)
        id_list = []
        features = glob.glob(
            os.path.join(self.features_folder, "**", "*.npy"), recursive=True
        )


        with env.begin(write=True) as txn:
            for infile in tqdm.tqdm(features):
                reader = np.load(infile, allow_pickle=True)
                item = {}
                split = os.path.relpath(infile, self.features_folder).split(
                    ".npy"
                )[0]
                item["feature_path"] = split
                key = split.encode()
                id_list.append(key)

                item["features"] = reader.item().get("features")
                item["image_height"] = reader.item().get("image_height")
                item["image_width"] = reader.item().get("image_width")
                item["num_boxes"] = reader.item().get("num_boxes")
                item["objects"] = reader.item().get("objects")
                item["cls_prob"] = reader.item().get("cls_prob", None)
                item["bbox"] = reader.item().get("bbox")

                txn.put(key, pickle.dumps(item))

            txn.put(b"keys", pickle.dumps(id_list))


In [ ]:
features_folder = '/content/features/'
lmdb_path = "/content/"
lmdb_converter = LMDBConversion(features_folder, lmdb_path)
lmdb_converter.convert()

100%|██████████| 16522/16522 [01:30<00:00, 181.86it/s]


#### <font color='PaleGoldenrod'> <b> Convert .lmdb to .npy </b> </font>
just to check if everything's okay

In [ ]:
features_folder = "/content/features_from_lmdb/"
lmdb_path = "/content/drive/MyDrive/"

def extract():
    os.makedirs(features_folder, exist_ok=True)
    env = lmdb.open(
        lmdb_path,
        max_readers=1,
        readonly=True,
        lock=False,
        readahead=False,
        meminit=False,
    )
    with env.begin(write=False) as txn:
        _image_ids = pickle.loads(txn.get(b"keys"))
        for img_id in tqdm.tqdm(_image_ids):
            item = pickle.loads(txn.get(img_id))
            img_id = img_id.decode("utf-8")
            
            tmp_dict = {
                "image_id"    : img_id,
                "bbox"        : item["bbox"],
                "num_boxes"   : item["num_boxes"],
                "image_height": item["image_height"],
                "image_width" : item["image_width"],
                "objects"     : item["objects"],
                "cls_prob"    : item["cls_prob"],
            }

            info_file_base_name = str(img_id) + "_info.npy"
            file_base_name = str(img_id) + ".npy"

            np.save(
                os.path.join(features_folder, file_base_name),
                item["features"],
            )
            np.save(
                os.path.join(features_folder, info_file_base_name),
                tmp_dict,
            )

In [ ]:
extract()

100%|██████████| 16522/16522 [04:24<00:00, 62.54it/s] 


In [ ]:
data = np.load("/content/features_from_lmdb/01243.npy", allow_pickle=True)
data_info = np.load("/content/features_from_lmdb/01243_info.npy", allow_pickle=True)

In [ ]:
data_info.item()

In [ ]:
data.shape

### <font color='Gold'><b> Number of 'unique' (based on file names) images </b></font>


In [ ]:
import pandas as pd
import os

annotation_dir = "/root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/annotations"
img_dir = "/root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/img/"

# Collect all the annotations (from Phase-2)
train       = pd.read_json(f"{annotation_dir}/train.jsonl", lines=True)
dev_seen    = pd.read_json(f"{annotation_dir}/dev_seen.jsonl", lines=True)
dev_unseen  = pd.read_json(f"{annotation_dir}/dev_unseen.jsonl", lines=True)
test_seen   = pd.read_json(f"{annotation_dir}/test_seen.jsonl", lines=True)
test_unseen = pd.read_json(f"{annotation_dir}/test_unseen.jsonl", lines=True)

# Create 2 sets: 
#   A set of strings, 'a': for all the image names,
#   A set of lists, 'b': for all the image names in dataset, e.g. train, dev_seen, etc.
a = os.listdir(f"{img_dir}")
b = []
for i in [train, dev_seen, dev_unseen, test_seen, test_unseen]:
    b.append(list(i["img"].str.split("/").str.get(1)))

set_mapping = ['train', 'dev_seen', 'dev_unseen', 'test_seen', 'test_unseen']
total_size = 0
print("#of images in: ")
for idx, i in enumerate(b):
    total_size += len(set(i))
    print(f"\t'{set_mapping[idx]}'  \t:", len(set(i)))
else:
    print(f"\nIn total there are {total_size} images,",
          "\nBut the # of images in /img/ directory is: ", len(a))

In [ ]:
# First, let's check if all the images are within jsonl files, in other words: 
# 'do we have an image in /img folder that's not in one of the .jsonl files?'
# 0 means every image in /img directory is in a jsonl file
print("#of images that are not in one of the .jsonl files: ", 
      len(set(a).symmetric_difference(set(b[0] + b[1] + b[2] + b[3] + b[4]))))

In [ ]:
print("#of same images in between: ")
for i in range(0, 5):
    print("\n")
    for j in range(0, 5):
        if i != j:
            print(f"{set_mapping[i], set_mapping[j]}   \t: {len(set(b[i]) & set(b[j]))}")

As seen, `dev_seen.jsonl` and `dev_unseen.jsonl` have `400` same images. Let's double check that:

In [ ]:
print(f"#of same images in {set_mapping[1], set_mapping[2]}: {len(set(b[1]) & set(b[2]))}",
      f"\n#of different images: {len(set(b[1]).symmetric_difference(set(b[2])))}")

That means in Phase-2, `100` images were removed from `dev_seen.jsonl` and `140` new images are added to the validation set.\
Hence;  `|dev_unseen.jsonl|=500-100+140=540`

### <font color='Gold'> <b> Image Feature Extraction </b> </font>

#### <font color='PaleGoldenrod'> <b> Discovering default image features from MMF </b> </font>

Download the features for Phase-2 which was published by MMF on 01.10.2020 [[source of link]](https://github.com/facebookresearch/mmf/blob/518a5a675586e4dc1b415a52a8a80c75edfc2960/mmf/configs/zoo/datasets.yaml#L232)

In [ ]:
# Download the features for Phase-2 from the following link
!wget https://dl.fbaipublicfiles.com/mmf/data/datasets/hateful_memes/defaults/features/features_2020_10_01.tar.gz

Extract `features_2020_10_01.tar.gz`:
> this file actually extracts a folder `detectron.lmdb/` which
stores the `data.mdb` file, which is where all the image features are compressed into.

In [ ]:
!tar -xzf /content/features_2020_10_01.tar.gz

Extract image features from `detectron.lmdb/` folder to `/features/`.

**Note**
Interrupt the execution as we only want to have a sneek peek into a few features. So, no need to extract the whole image features.

In [ ]:
!python /content/mmf/tools/scripts/features/lmdb_conversion.py \
        --mode "extract" \
        --lmdb_path "/content/detectron.lmdb" \
        --features_folder "/content/features/" \

In [ ]:
# Load only one image feature
import numpy as np
import os

img_list = os.listdir("/content/features/")
feat_dir = "/content/features/"


if len(img_list[0].split('_'))==2:
    img_id = img_list[0].split('_')[0]
else:
    img_id = img_list[0].split('_')[0].split('.')[0]

# There are 2 .npy files for each image, 
#   e.g. : for the image with 'image_id=75349':
#       - '75349.npy' : actual feature embedding
#       - '75349_info.npy' : meta-data about the image
data      = np.load(f"{feat_dir + img_id}.npy", allow_pickle=True)
data_info = np.load(f"{feat_dir + img_id}_info.npy", allow_pickle=True)

In [ ]:
# Images are embedded to 2048 dimension!
# There are 100 bbox's
data.shape

In [ ]:
# The meta-data about the image
data_info.item().keys()

In [ ]:
print(f"image_id\t: {data_info.item()['image_id']}",
      f"\nnum_boxes\t: {data_info.item()['num_boxes']}",
      f"\nimage_height\t: {data_info.item()['image_height']}",
      f"\nimage_width\t: {data_info.item()['image_width']}",
      f"\nshape(bbox)\t: {data_info.item()['bbox'].shape}",
      f"\nshape(objects)\t: {data_info.item()['objects'].shape}",
      f"\nshape(cls_prob)\t: {data_info.item()['cls_prob'].shape}")

In [ ]:
data_info.item()["objects"]

In [ ]:
for i in [0, 1, 2]:
  print(f"max cls_prob of box #{[i]}: {data_info.item()['cls_prob'][i].max()}",
        f"\nindex of that class\t: {data_info.item()['cls_prob'][i].argmax()}\n",
        "-"*10)

In [ ]:
data_info.item()["bbox"]

#### <font color='PaleGoldenrod'> <b> Different techniques for image feature extraction </b> </font>

##### <font color='PeachPuff'> <b> Extract image features using `Detectron2` & `ResNet-152` </b> </font>

In [ ]:
!python extract_region_feature.py

  0% 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/detectron2/modeling/poolers.py:231: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  inds = torch.nonzero(level_assignments == level).squeeze(1)
100% 3/3 [00:20<00:00,  6.84s/it]


In [ ]:
data = np.load("/content/features/hateful_memes/image_1.npy", allow_pickle=True)
data.item(0)

In [ ]:
data.item(0)["features"].shape

In [ ]:
data.item().keys()

##### <font color='PeachPuff'> <b> Extract image features using [`facebookresearch/grid-feats-vqa`](https://github.com/facebookresearch/grid-feats-vqa) </b> </font>

In [ ]:
# Install required packages
!pip install -U git+https://github.com/facebookresearch/fvcore
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git@ffff8ac'

In [ ]:
!git clone https://github.com/vedanuj/grid-feats-vqa.git --branch region_features

In [ ]:
cd grid-feats-vqa/

/content/grid-feats-vqa


In [ ]:
!python extract_region_feature.py \
              --config-file configs/X-152-region-c4.yaml \
              --dataset "hateful_memes" \
              --dataset-path "/root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/img/" \

In [ ]:
!ls /content/grid-feats-vqa/output/features/hateful_memes/

In [ ]:
import numpy as np
data = np.load("/content/grid-feats-vqa/output/features/hateful_memes/01235.npy", allow_pickle=True)

In [ ]:
data.item(0)

In [ ]:
data.item(0)["bbox"].shape

(9, 4)

##### <font color='PeachPuff'> <b> Extract image features using [`airsplay/py-bottom-up-attention`](https://github.com/airsplay/py-bottom-up-attention) </b> </font>

###### **Install packages**

In [ ]:
import os
os.chdir("/content/")
!git clone https://github.com/airsplay/py-bottom-up-attention.git
os.chdir("py-bottom-up-attention/")

Cloning into 'py-bottom-up-attention'...
remote: Enumerating objects: 1991, done.
remote: Total 1991 (delta 0), reused 0 (delta 0), pack-reused 1991
Receiving objects: 100% (1991/1991), 8.94 MiB | 34.95 MiB/s, done.
Resolving deltas: 100% (1225/1225), done.


In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
# Install detectron2
!python setup.py build develop

###### **Extract!**

In [ ]:
os.chdir("/content/hateful_memes/region_feature_extraction/")
!python extract.py

In [ ]:
!ls /content/features/ | wc -l

*What is the size of the training data?*
> 

*Did you use additional dataset?*
> Yes. I used Memotion as an additional dataset

*Did you use pre-trained models?*
> Yes. I used `VisualBERT` which was pre-trained on `Masked Conceptual Captions` dataset, see <font color='magenta'> <b> IV. Fine-tuning pre-trained VisualBERT models on Hateful Memes </b> </font>. Then, the model was fine-tuned on the HM dataset. The pre-trained model is available from MMF: [See all the available pre-trained VisualBERT models from MMF](https://github.com/facebookresearch/mmf/tree/master/projects/pretrain_vl_right)

*Did you use default image features provided by MMF?*
> No. I extracted our own image features using Facebook's Detectron model, which uses ResNet-152 as its backbone. See <font color='magenta'> <b> III. Feature Extraction </b> </font> part in the notebook.

*What is the impact of `Majority Voting` technique in ROC-AUC score? And what do you think about the reason behind?*
>

**
>